In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os
import shutil
from pathlib import Path

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PHOTO_PATH = "../input/gan-getting-started/photo_jpg"
MODEL_PATH = "../input/my-monet-cyclegan-weights1/epoch_8.pth" 

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)

class GeneratorResNet(nn.Module):
    def __init__(self, num_residual_blocks=9):
        super(GeneratorResNet, self).__init__()
        
        out_features = 64
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(3, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True)
        ]
        in_features = out_features

        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features

        for _ in range(num_residual_blocks):
            model += [ResidualBlock(in_features)]

        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features

        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, 7),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalize=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(3, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)

In [4]:
class PhotoDataset(Dataset):
    def __init__(self, root):
        self.files = [os.path.join(root, f) for f in os.listdir(root) if f.endswith(".jpg")]
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

    def __getitem__(self, index):
        img_path = self.files[index]
        img = Image.open(img_path).convert('RGB')
        return self.transform(img), os.path.basename(img_path)

    def __len__(self):
        return len(self.files)

In [ ]:

def make_submission():
    G_AB = GeneratorResNet().to(DEVICE)
    
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    G_AB.load_state_dict(checkpoint['G_AB'])
    
    G_AB.eval()
    print("Model loaded!")

    dataset = PhotoDataset(PHOTO_PATH)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    os.makedirs("../images", exist_ok=True)

    print(f"Generating images for {len(dataset)} photos...")
    
    for i, (imgs, filenames) in enumerate(dataloader):
        imgs = imgs.to(DEVICE)
        
        with torch.no_grad():
            fake_monets = G_AB(imgs)
        
        fake_monets = fake_monets * 0.5 + 0.5
        
        for j in range(len(fake_monets)):
            save_path = os.path.join("../images", filenames[j])
            save_image(fake_monets[j], save_path)
            
        if i % 500 == 0:
            print(f"Processed {i} batches...")

    print("Zipping images...")
    shutil.make_archive("/kaggle/working/images", 'zip', "../images")
    
    shutil.rmtree("../images")
    print("Done! images.zip is ready.")

if __name__ == "__main__":
    make_submission()

Model loaded!
Generating images for 7038 photos...
Processed 0 batches...
Processed 500 batches...
Processed 1000 batches...
Processed 1500 batches...
Processed 2000 batches...
Processed 2500 batches...
Processed 3000 batches...
Processed 3500 batches...
Processed 4000 batches...
Processed 4500 batches...
Processed 5000 batches...
Processed 5500 batches...
Processed 6000 batches...
Processed 6500 batches...
Processed 7000 batches...
Zipping images...
Done! images.zip is ready.
